# Phase 2: Quick Baseline（快速基线）

---

## 📋 本阶段目标

在Phase 1的数据诊断基础上，**快速建立多个基线模型**，为后续优化提供性能基准。

### 核心任务

1. ✅ **快速数据预处理**：使用简化的预处理流程
2. ✅ **数据集划分**：训练集、验证集、测试集
3. ✅ **训练多个基线模型**：Logistic Regression、Random Forest、XGBoost、LightGBM
4. ✅ **性能对比**：交叉验证评估
5. ✅ **选择最佳基线**：确定后续优化的起点

### 方法论

- **简单优先**：使用默认参数，不做过度调优
- **快速迭代**：快速获得性能基准，识别问题
- **多样性**：尝试不同类型的算法（线性、树模型、集成）
- **可重复性**：固定随机种子，确保结果可复现

---

## 预期时间

⏱️ **10-30分钟**（取决于数据集大小）

---

## 1. 环境准备

In [ ]:
# 导入必要的库
import sys
import warnings
from pathlib import Path

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# 导入本项目的模块
from src import data_preprocessing as dp
from src import supervised_pipeline as sp
from src import model_evaluation as me
from src import visualization as viz

# 设置
warnings.filterwarnings('ignore')
sns.set_style('whitegrid')
plt.rcParams['font.sans-serif'] = ['Arial Unicode MS', 'SimHei', 'DejaVu Sans']
plt.rcParams['axes.unicode_minus'] = False

# 随机种子
RANDOM_STATE = 42
np.random.seed(RANDOM_STATE)

print("✅ 环境准备完成")
print(f"当前工作目录: {Path.cwd()}")

---

## 2. 数据加载

加载原始数据或Phase 1处理后的数据

In [ ]:
# 方式1: 加载原始数据（示例：Titanic）
# 你可以替换为自己的数据集

# 使用Titanic数据集作为演示
df = sns.load_dataset('titanic')

# 定义目标变量
target_col = 'survived'

print(f"数据形状: {df.shape}")
print(f"\n目标变量: {target_col}")
print(f"目标变量分布:\n{df[target_col].value_counts()}")

# 快速查看数据
df.head()

In [ ]:
# 方式2: 如果你在Phase 1保存了清洗后的数据，可以这样加载
# processed_data_path = Path('data/processed/cleaned_data.csv')
# if processed_data_path.exists():
#     df = pd.read_csv(processed_data_path)
#     print(f"✅ 加载Phase 1处理后的数据: {df.shape}")

---

## 3. 快速数据预处理

使用 `quick_preprocess()` 函数快速完成基本预处理：

### 处理步骤

1. 分离目标变量
2. 删除ID列（唯一值比例>95%）
3. 删除常量列
4. 删除重复行
5. 删除缺失率过高的列（默认>50%）
6. 简单填充剩余缺失值（数值型用中位数，类别型用众数）
7. One-Hot编码类别特征（限制≤20类）
8. 标准化数值特征

In [ ]:
# 快速预处理
print("="*60)
print("开始快速预处理...")
print("="*60)

X, y = dp.quick_preprocess(
    df, 
    target_col=target_col,
    drop_missing_threshold=0.5,  # 删除缺失率>50%的列
    handle_categorical=True,      # 处理类别特征
    scale_features=True           # 标准化数值特征
)

print("\n" + "="*60)
print("预处理完成！")
print("="*60)
print(f"特征矩阵形状: {X.shape}")
print(f"目标变量形状: {y.shape}")
print(f"\n特征列表 (前10个):")
print(X.columns.tolist()[:10])

In [ ]:
# 检查预处理结果
print("预处理质量检查:")
print(f"- 缺失值总数: {X.isnull().sum().sum()}")
print(f"- 重复行数: {X.duplicated().sum()}")
print(f"- 数据类型分布:\n{X.dtypes.value_counts()}")

---

## 4. 数据集划分

划分为训练集(70%)、验证集(15%)、测试集(15%)

In [ ]:
# 数据集划分
X_train, X_val, X_test, y_train, y_val, y_test = dp.split_data(
    X, y,
    train_size=0.7,
    val_size=0.15,
    test_size=0.15,
    random_state=RANDOM_STATE,
    stratify=True  # 保持目标变量分布一致
)

print("数据集划分完成:")
print(f"  训练集: {X_train.shape}")
print(f"  验证集: {X_val.shape}")
print(f"  测试集: {X_test.shape}")

# 检查目标变量分布
print("\n目标变量分布:")
print(f"  训练集: {y_train.value_counts(normalize=True).to_dict()}")
print(f"  验证集: {y_val.value_counts(normalize=True).to_dict()}")
print(f"  测试集: {y_test.value_counts(normalize=True).to_dict()}")

---

## 5. 建立基线模型

### 5.1 确定问题类型

In [ ]:
# 自动检测问题类型
unique_values = y.nunique()

if unique_values <= 20:  # 假设类别数≤20为分类问题
    problem_type = 'classification'
    print(f"✅ 检测到分类问题 (类别数: {unique_values})")
else:
    problem_type = 'regression'
    print(f"✅ 检测到回归问题")

### 5.2 训练多个基线模型

使用 `SupervisedPipeline` 快速训练4种不同的模型：
- **Logistic Regression** (线性模型，可解释性强)
- **Random Forest** (集成树模型，性能稳定)
- **XGBoost** (梯度提升树，通常性能优秀)
- **LightGBM** (快速梯度提升树，适合大数据)

In [ ]:
# 创建监督学习Pipeline
pipeline = sp.SupervisedPipeline(
    problem_type=problem_type,
    random_state=RANDOM_STATE
)

# 训练多个模型（5折交叉验证）
pipeline.fit(X_train, y_train, cv=5)

### 5.3 模型性能对比

In [ ]:
# 获取模型对比结果
comparison_df = pipeline.get_model_comparison()

print("\n" + "="*60)
print("交叉验证性能对比")
print("="*60)
print(comparison_df.to_string(index=False))
print("\n注: 分数为5折交叉验证的平均值")

In [ ]:
# 可视化模型性能对比
fig, ax = plt.subplots(figsize=(10, 6))

# 绘制条形图
bars = ax.barh(
    comparison_df['模型'], 
    comparison_df['平均得分'],
    xerr=comparison_df['标准差'],
    capsize=5,
    color='skyblue',
    edgecolor='navy',
    alpha=0.7
)

# 高亮最佳模型
best_idx = comparison_df['平均得分'].idxmax()
bars[best_idx].set_color('orange')
bars[best_idx].set_edgecolor('darkred')
bars[best_idx].set_linewidth(2)

ax.set_xlabel('交叉验证平均得分', fontsize=12)
ax.set_title('基线模型性能对比 (5-Fold CV)', fontsize=14, fontweight='bold')
ax.grid(axis='x', alpha=0.3)

# 添加数值标签
for i, (score, std) in enumerate(zip(comparison_df['平均得分'], comparison_df['标准差'])):
    ax.text(score + 0.01, i, f'{score:.4f} ± {std:.4f}', 
            va='center', fontsize=10)

plt.tight_layout()
plt.show()

print(f"\n✅ 最佳基线模型: {pipeline.best_model_name}")

---

## 6. 验证集评估

使用验证集对最佳模型进行更详细的评估

In [ ]:
# 在验证集上预测
y_val_pred = pipeline.predict(X_val)

if problem_type == 'classification':
    y_val_proba = pipeline.predict_proba(X_val)
    
    # 分类评估
    print("="*60)
    print(f"验证集评估 - {pipeline.best_model_name}")
    print("="*60)
    
    # 计算评估指标
    val_metrics = me.evaluate_classification(
        y_val, 
        y_val_pred,
        y_val_proba[:, 1] if y_val_proba.shape[1] == 2 else None,
        verbose=True
    )
    
else:  # regression
    # 回归评估
    print("="*60)
    print(f"验证集评估 - {pipeline.best_model_name}")
    print("="*60)
    
    val_metrics = me.evaluate_regression(
        y_val,
        y_val_pred,
        verbose=True
    )

### 6.1 混淆矩阵（分类问题）

In [ ]:
if problem_type == 'classification':
    # 绘制混淆矩阵
    me.plot_confusion_matrix(y_val, y_val_pred)
    plt.show()

### 6.2 ROC曲线（二分类问题）

In [ ]:
if problem_type == 'classification' and y.nunique() == 2:
    # 绘制ROC曲线
    me.plot_roc_curve(y_val, y_val_proba[:, 1])
    plt.show()

### 6.3 预测结果可视化（回归问题）

In [ ]:
if problem_type == 'regression':
    # 绘制预测vs实际
    me.plot_regression_results(y_val, y_val_pred)
    plt.show()

---

## 7. 测试集最终评估

**注意**: 测试集只在最后评估一次，避免过拟合到测试集

In [ ]:
# 在测试集上预测
y_test_pred = pipeline.predict(X_test)

if problem_type == 'classification':
    y_test_proba = pipeline.predict_proba(X_test)
    
    # 分类评估
    print("="*60)
    print(f"测试集最终评估 - {pipeline.best_model_name}")
    print("="*60)
    
    test_metrics = me.evaluate_classification(
        y_test,
        y_test_pred,
        y_test_proba[:, 1] if y_test_proba.shape[1] == 2 else None,
        verbose=True
    )
    
    # 混淆矩阵
    me.plot_confusion_matrix(y_test, y_test_pred)
    plt.show()
    
else:  # regression
    # 回归评估
    print("="*60)
    print(f"测试集最终评估 - {pipeline.best_model_name}")
    print("="*60)
    
    test_metrics = me.evaluate_regression(
        y_test,
        y_test_pred,
        verbose=True
    )
    
    # 预测结果可视化
    me.plot_regression_results(y_test, y_test_pred)
    plt.show()

---

## 8. 性能总结

对比训练、验证、测试三个集合的性能

In [ ]:
# 汇总性能指标
if problem_type == 'classification':
    # 获取训练集性能（交叉验证）
    train_score = pipeline.models[pipeline.best_model_name]['mean_score']
    
    summary = pd.DataFrame({
        '数据集': ['训练集(CV)', '验证集', '测试集'],
        'Accuracy': [
            train_score,
            val_metrics.get('accuracy', None),
            test_metrics.get('accuracy', None)
        ],
        'Precision': [
            None,
            val_metrics.get('precision', None),
            test_metrics.get('precision', None)
        ],
        'Recall': [
            None,
            val_metrics.get('recall', None),
            test_metrics.get('recall', None)
        ],
        'F1-Score': [
            None,
            val_metrics.get('f1', None),
            test_metrics.get('f1', None)
        ]
    })
else:  # regression
    train_score = pipeline.models[pipeline.best_model_name]['mean_score']
    
    summary = pd.DataFrame({
        '数据集': ['训练集(CV)', '验证集', '测试集'],
        'MSE': [
            -train_score,  # 注意：sklearn的neg_mean_squared_error
            val_metrics.get('mse', None),
            test_metrics.get('mse', None)
        ],
        'RMSE': [
            np.sqrt(-train_score),
            val_metrics.get('rmse', None),
            test_metrics.get('rmse', None)
        ],
        'MAE': [
            None,
            val_metrics.get('mae', None),
            test_metrics.get('mae', None)
        ],
        'R²': [
            None,
            val_metrics.get('r2', None),
            test_metrics.get('r2', None)
        ]
    })

print("\n" + "="*60)
print(f"Phase 2 性能总结 - {pipeline.best_model_name}")
print("="*60)
print(summary.to_string(index=False))
print("\n注: CV = 5折交叉验证")

---

## 9. Phase 2 总结与建议

### 9.1 当前基线性能

In [ ]:
print("\n" + "="*60)
print("📊 Phase 2 总结")
print("="*60)

print(f"\n✅ 最佳基线模型: {pipeline.best_model_name}")
print(f"\n📈 测试集性能:")
if problem_type == 'classification':
    print(f"   - Accuracy:  {test_metrics['accuracy']:.4f}")
    print(f"   - Precision: {test_metrics['precision']:.4f}")
    print(f"   - Recall:    {test_metrics['recall']:.4f}")
    print(f"   - F1-Score:  {test_metrics['f1']:.4f}")
else:
    print(f"   - RMSE:      {test_metrics['rmse']:.4f}")
    print(f"   - MAE:       {test_metrics['mae']:.4f}")
    print(f"   - R² Score:  {test_metrics['r2']:.4f}")

print(f"\n📋 数据集信息:")
print(f"   - 训练集: {X_train.shape[0]} 样本")
print(f"   - 验证集: {X_val.shape[0]} 样本")
print(f"   - 测试集: {X_test.shape[0]} 样本")
print(f"   - 特征数: {X_train.shape[1]} 个")

### 9.2 后续优化方向

根据Phase 2的基线结果，以下是**Phase 3: Supervised Solution**的优化方向：

In [ ]:
print("\n" + "="*60)
print("🎯 Phase 3 优化建议")
print("="*60)

optimization_suggestions = [
    "1. 特征工程",
    "   - 创建交互特征（特征组合）",
    "   - 多项式特征（非线性关系）",
    "   - 特征选择（去除冗余特征）",
    "   - 领域知识驱动的特征构造",
    "",
    "2. 数据预处理优化",
    "   - 高级缺失值填充（KNN、迭代填充）",
    "   - 高级编码（Target Encoding、Frequency Encoding）",
    "   - 异常值处理（Winsorization、离群点移除）",
    "   - 不平衡数据处理（SMOTE、类别权重）",
    "",
    "3. 模型优化",
    f"   - 超参数调优（{pipeline.best_model_name}）",
    "   - 模型集成（Voting、Stacking）",
    "   - 尝试更多算法（神经网络、SVM等）",
    "",
    "4. 模型诊断",
    "   - 学习曲线分析（是否过拟合/欠拟合）",
    "   - 错误案例分析（哪些样本预测错误）",
    "   - 特征重要性分析（哪些特征最有用）",
]

for suggestion in optimization_suggestions:
    print(suggestion)

### 9.3 Phase 2 检查清单

In [ ]:
print("\n" + "="*60)
print("✅ Phase 2 完成检查清单")
print("="*60)

checklist = [
    "✅ 数据加载成功",
    "✅ 快速预处理完成（无缺失值、无重复行）",
    "✅ 数据集正确划分（训练/验证/测试）",
    "✅ 训练了多个基线模型（≥3个）",
    "✅ 模型性能对比完成",
    "✅ 选择了最佳基线模型",
    "✅ 验证集评估完成",
    "✅ 测试集最终评估完成",
    "✅ 性能可视化完成",
    "✅ Phase 3优化方向明确"
]

for item in checklist:
    print(f"  {item}")

print("\n" + "="*60)
print("🎉 Phase 2: Quick Baseline 完成！")
print("="*60)
print(f"\n下一步: 前往 phase3_supervised_solution.ipynb 进行深度优化")

---

## 10. 保存模型和结果（可选）

In [ ]:
# 保存最佳模型
# import joblib
# model_path = Path('models/phase2_baseline_model.pkl')
# model_path.parent.mkdir(parents=True, exist_ok=True)
# joblib.dump(pipeline.best_model, model_path)
# print(f"✅ 模型已保存到: {model_path}")

# 保存性能总结
# results_path = Path('results/phase2_baseline_results.csv')
# results_path.parent.mkdir(parents=True, exist_ok=True)
# summary.to_csv(results_path, index=False)
# print(f"✅ 结果已保存到: {results_path}")